In [40]:
from mealpy import Problem, FloatVar, BBO, IntegerVar, GA, StringVar
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from mealpy import GA, PSO, Problem, FloatVar, IntegerVar
import numpy as np

SEED = 42


# Our custom problem class
class Squared(Problem):
    def __init__(self, bounds=None, minmax="min", name="Squared", data=None, **kwargs):
        self.name = name
        self.data = data
        super().__init__(bounds, minmax, **kwargs)

    def obj_func(self, solution):
        x = self.decode_solution(solution)["weight"]
        b = self.decode_solution(solution)["base"]
        return [x**2, b**2]

## Now, we define an algorithm, and pass an instance of our *Squared* class as the problem argument.
bounds = [IntegerVar(lb=10, ub=100, name="weight"), IntegerVar(lb=10, ub=20, name="base")]
problem = Squared(bounds=bounds, minmax="min", name="Squared", data="Amazing", obj_weights=(0.5, 0.5))
# model = BBO.OriginalBBO(epoch=100, pop_size=20, seed=SEED)
# model_2 = GA.BaseGA(epoch=100, pop_size=100)
# g_best = model.solve(problem)

foo


In [8]:
class LogisticRegression():
    def __init__(self, input: int, optimizer: str, learning_rate: float):
        self.input = input
        self.optimizer_name = optimizer
        self.learning_rate = learning_rate
        self.sequential = Sequential([
            Input(shape=(input,)),
            Dense(1, activation='linear'),
        ])

    def fit(self, X, y, epochs=50):
        self.sequential.compile(
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=getattr(tf.keras.optimizers, self.optimizer_name)(learning_rate=self.learning_rate))
        self.sequential.fit(X, y, epochs=epochs)
        loss = self.sequential.evaluate(X, y)
        return loss

    def predict(self, X, y):
        test_loss = self.sequential.evaluate(X, y)
        return test_loss



In [37]:
class HyperParamProblem(Problem):
    def __init__(self, bounds, minmax="min", data=None, **kwargs):
        self.data = data

        super().__init__(bounds, minmax, **kwargs)

    # def fit_model(self, optimizer_name, learning_rate):
    #     # Create Logistic Regression model with current hyperparameters
    #     input_dim = self.data["X_train"].shape[1]
    #     model = LogisticRegression(input=input_dim, optimizer=optimizer_name, learning_rate=learning_rate)

    #     # Train the model and return loss as the evaluation metric
    #     loss_train = model.fit(self.data["X_train"], self.data["y_train"], epochs=self.data["epoch"])
    #     loss_test = model.predict(self.data["X_test"], self.data["y_test"])
    #     return [loss_train, loss_test]

    def obj_func(self, solution):
        solution = self.decode_solution(solution)
        optimizer_name, learning_rate = solution["optimizer"], solution["learning_rate"]
        # train_test_loss = self.fit_model(optimizer_name, learning_rate)
        print("foo")
        return True

In [9]:
data = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.33, random_state=42)

In [38]:
data = {
    "X_train": X_train,
    "y_train": y_train,
    "X_test": X_test,
    "y_test": y_test,
    "epoch": 50
}
optimizers = [ "SGD", "Adam", "RMSprop", "Adagrad", "Adadelta", "Adamax"]
bounds = [
    StringVar(valid_sets=optimizers, name="optimizer"),
    FloatVar(lb=0.001, ub=0.1, name="learning_rate"),
]

In [39]:
hyperparam_problem = HyperParamProblem(bounds=bounds, minmax="min", data=data, name="Hyperparam_problem", obj_weights=(0.5, 0.5))

foo
